In [ ]:
#发送请求

In [7]:
APIURL = "https://c15xxx.execute-api.us-east-1.amazonaws.com/prod"

In [17]:
!curl -X POST -H 'Content-Type: application/json' $APIURL --data '@./payload1.json' 

{"id_task": "99808", "task": "text-to-image", "sd_model_checkpoint": "revAnimated_v122.safetensors", "output_location": "s3://sdoneksdataplanestack-outputs3bucket9fe85b9f-pzlnjnu6xsl7/99808"}

In [ ]:
#查看结果

In [13]:
s3uri = "s3://sdoneksdataplanestack-outputs3bucket9fe85b9f-pzlnjnu6xsl7/99808"

In [18]:
!aws s3 ls $s3uri --recursive
!aws s3 rm $s3uri/response.out

2023-09-09 03:51:24     415848 99808/20230909035123-16e72a3b-e6f3-4d39-a06f-4828e4b6a78e.png
2023-09-09 06:38:02     397862 99808/20230909063801-77a0793e-168b-4cf8-8d60-35b1223b9bd0.png
2023-09-09 06:44:35     386110 99808/20230909064434-90754b87-83e3-453f-840a-9b25f2ffcbc2.png
2023-09-09 06:44:35        279 99808/response.out


In [14]:
#查看最新生成的图片

In [ ]:
import boto3
import urllib
from PIL import Image
import io

s3_resource = boto3.resource('s3')
def get_latest_object_uri(s3_uri):
    bucket_name, prefix = parse_s3_uri(s3_uri)
    
    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
    
    if 'Contents' in response:
        objects = response['Contents']
        objects.sort(key=lambda obj: obj['LastModified'], reverse=True)
        latest_object = objects[0]
        return f"s3://{bucket_name}/{latest_object['Key']}"
    
    return None

def _get_output_bucket_key(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    return bucket, key

def parse_s3_uri(s3_uri):
    parts = s3_uri.replace('s3://', '').split('/')
    bucket_name = parts[0]
    prefix = '/'.join(parts[1:])
    return bucket_name, prefix

# Gen ObjectURI
s3_uri = s3uri
latest_object_uri = get_latest_object_uri(s3_uri)
if latest_object_uri:
    print(latest_object_uri)
else:
    print("No objects found.")
    
bucket, key = _get_output_bucket_key(latest_object_uri)
obj = s3_resource.Object(bucket, key)
bytes = obj.get()['Body'].read()
image = Image.open(io.BytesIO(bytes))
image.show()


In [10]:
#查看最近生成的随机的图片

In [ ]:
import boto3
import random
from PIL import Image
import requests
import io
import urllib
from io import BytesIO

def get_latest_images(s3_uri, num_images):
    bucket_name, prefix = parse_s3_uri(s3_uri)

    s3 = boto3.client('s3')
    response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)

    if 'Contents' in response:
        objects = response['Contents']
        objects.sort(key=lambda obj: obj['LastModified'], reverse=True)
        latest_objects = objects[:num_images]
        image_uris = [f"s3://{bucket_name}/{obj['Key']}" for obj in latest_objects]
        return image_uris

    return []

def _get_output_bucket_key(output_location):
    output_url = urllib.parse.urlparse(output_location)
    bucket = output_url.netloc
    key = output_url.path[1:]
    return bucket, key

def parse_s3_uri(s3_uri):
    parts = s3_uri.replace('s3://', '').split('/')
    bucket_name = parts[0]
    prefix = '/'.join(parts[1:])
    return bucket_name, prefix

# Example usage
s3_uri = s3uri
num_images = 5

latest_image_uris = get_latest_images(s3_uri, num_images)
if latest_image_uris:
    random_image_uri = random.choice(latest_image_uris)
    print("Randomly selected image URI:", random_image_uri)
    

    bucket, key = _get_output_bucket_key(random_image_uri)
    obj = s3_resource.Object(bucket, key)
    bytes = obj.get()['Body'].read()
    image = Image.open(io.BytesIO(bytes))
    image.show()
else:
    print("No images found.")
